<a href="https://colab.research.google.com/github/hanene2030/pandas/blob/main/13_grouping_aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grouping aggregation

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("train.csv" , low_memory=False, parse_dates=["Date"])
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555.0,1.0,1.0,0,1.0
1,2,5,2015-07-31,6064,625.0,1.0,1.0,0,1.0
2,3,5,2015-07-31,8314,821.0,1.0,1.0,0,1.0
3,4,5,2015-07-31,13995,1498.0,1.0,1.0,0,1.0
4,5,5,2015-07-31,4822,559.0,1.0,1.0,0,1.0


# Single aggregates

In [5]:
df.groupby("Store").median().head(3)

,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
Store,,,,,,
1,4.0,4211.0,511.0,1.0,0.0,0.0
2,4.0,4435.0,539.0,1.0,0.0,0.0
3,4.0,5868.0,699.0,1.0,0.0,0.0


In [6]:
## Different aggregates for different columns

In [7]:
df.groupby(["Store", "DayOfWeek"]).agg

<bound method DataFrameGroupBy.aggregate of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7f623c3e6e90>>

In [8]:
df.groupby(["Store", "DayOfWeek"]).agg({"Sales": "mean", "Customers": "count"})

Sales  Customers
Store DayOfWeek                        
1     1          4780.868421         76
      2          4657.171053         76
      3          4462.973684         76
      4          3949.960526         76
      5          4272.207792         77
...                      ...        ...
1115  3          6293.526316         76
      4          5736.631579         76
      5          6568.302632         76
      6          7327.881579         76
      7             0.000000         76

[7805 rows x 2 columns]

In [11]:
df2 = df.groupby(["Store", "DayOfWeek"]).agg({"Sales": ["mean", "max", "min"], "Customers": "count"})
df2.head(2)

Sales             Customers
                        mean   max   min     count
Store DayOfWeek                                   
1     1          4780.868421  9331     0        76
      2          4657.171053  7959  3037        76

In [13]:
df2.columns = ["SalesMean","SalesMax","SalesMin","Count"]
df2.head(2)

SalesMean  SalesMax  SalesMin  Count
Store DayOfWeek                                        
1     1          4780.868421      9331         0     76
      2          4657.171053      7959      3037     76

In [14]:
mc_uncert = lambda x: np.std(x)/np.sqrt(x.size)
df3 = df.groupby(["Store", "DayOfWeek"]).agg({"Sales": ["mean", mc_uncert], "Customers": "count"})
df3.head(2)

Sales             Customers
                        mean  <lambda_0>     count
Store DayOfWeek                                   
1     1          4780.868421  182.659091        76
      2          4657.171053  116.561919        76

In [15]:
mc_uncert = lambda x: np.std(x)/np.sqrt(x.size)
df3 = df.groupby(["Store", "DayOfWeek"]).agg({"Sales": [("SalesMean","mean"),("SalesUncert" , mc_uncert)], "Customers": "count"})
df3.head(2)

Sales             Customers
                   SalesMean SalesUncert     count
Store DayOfWeek                                   
1     1          4780.868421  182.659091        76
      2          4657.171053  116.561919        76

In [18]:
def mc_uncert2(x):
  return np.std(x)/np.sqrt(x.size)

df4 = df.groupby(["Store", "DayOfWeek"])
df4.agg(
      SalesMean=("Sales", "mean"),
      SalesUncert=("Sales", mc_uncert2)
  ).reset_index().head(2)

,Store,DayOfWeek,SalesMean,SalesUncert
0,1,1,4780.868421,182.659091
1,1,2,4657.171053,116.561919
